<a href="https://colab.research.google.com/github/hatefap/CrackingTheInterview/blob/master/roberto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip  install transformers numpy torch sklearn

     |████████████████████████████████| 2.0MB 7.3MB/s 
     |████████████████████████████████| 3.2MB 35.2MB/s 
     |████████████████████████████████| 890kB 53.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=bfafe55e2899b4ca0653611d4584d67d2dd84dffb469ea9dcc267914b1b1f901
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import XLMRobertaTokenizerFast, XLMRobertaForSequenceClassification
from transformers import Trainer, TrainingArguments
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd

In [ ]:
!pip install sentencepiece

In [ ]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(101)

In [ ]:
model_name = "xlm-roberta-base"
# max sequence length for each document/sentence sample
max_length = 512

In [ ]:
tokenizer_xlmroberta = XLMRobertaTokenizerFast.from_pretrained(model_name)

In [ ]:
farsi_path = "drive/MyDrive/final_final.csv"
english_path = "drive/MyDrive/my_final_english_hate.csv"
eng = pd.read_csv(english_path)
farsi = pd.read_csv(farsi_path)
eng.dropna(inplace=True)
farsi.dropna(inplace=True)

In [ ]:
farsi

,id,text,ref1,ref2,link,norm_text,google,mbart,ref3
0,0,بچه های عزیز این کشتزار منو بلاک کرده یکی که د...,1.0,0.0,https://twitter.com/DrErovay/status/1249849840...,بچه‌های عزیز این کشتزار منو بلاک کرده یکی که د...,Children's Menu block the field of view of one...,"My dear children, I've been blocked on this fa...",1.0
1,1,سیاسیون محترم از نمایندگان گرفته تا شهردارا...,0.0,0.0,https://twitter.com/atefeh heidari/status/1007...,سیاسیون محترم از نمایندگان گرفته تا شهرداران ...,Distinguished delegates politicians to munici...,From respectable politicians to elected offici...,0.0
2,2,با اعدام مخالفم \n\nکاش راهکاری برای نابودی و ...,1.0,0.0,https://twitter.com/fariborz c/status/10124857...,با اعدام مخالفم \n\nکاش راهکاری برای نابودی و ...,I disagree with execution\n\nIf only way to an...,"With the execution of my opponent, I wish ther...",1.0
3,3,دونوع آدم داریم : یکی اونکه هیچی به کیرش نیست ...,1.0,0.0,https://twitter.com/llil ili lill/status/10190...,دونوع آدم داریم: یکی اونکه هیچی به کیرش نیست و...,There are two types of people: one Avnkh nothi...,There are two types of people: There's the one...,0.0
4,4,هيچ زنداني بدتر از ،دختر بودن، نيست....,1.0,0.0,https://twitter.com/virgoOoOol/status/10233572...,هیچ زندانی بدتر از، دختر بودن، نیست ….,"No prisoners worse, the girl is not ....",There is no prison worse than being a girl...,0.0
...,...,...,...,...,...,...,...,...,...
5195,5196,من ایشون میشناسم تو اینستام پیج داره تا جایی ک...,0.0,0.0,https://twitter.com/Naslsokhte1/status/1150193...,من ایشون میشناسم تو اینستام پیج داره تا جایی ک...,She got as far as I know you saw Paige Aynstam...,I know one of them. He's been on this team for...,0.0
5196,5197,پسرا، هم جنسا، چرا دختر هرچی بددهن باشه و کص...,0.0,1.0,https://twitter.com/Merajsn/status/11501930125...,پسرا، هم جنسا، چرا دختر هرچی بددهن باشه و کصشع...,"Boys, both generically, because the girl is ge...","Guys, same-sex, why would a girl be anything b...",0.0
5197,5198,هدایت و بازداشت شده است.\nامارات سیاست سکوت را...,0.0,0.0,https://twitter.com/MadariSarzamin/status/1150...,هدایت و بازداشت شده است. \nامارات سیاست سکوت ر...,The guidance has been arrested.\nEmirates has ...,"The UAE has adopted a policy of silence, and s...",0.0
5198,5199,عزیزان، کسی گفته کاربران توئیتر مستجاب الدعوه ...,0.0,0.0,https://twitter.com/LeylaNarimani/status/11509...,عزیزان، کسی گفته کاربران توئیتر مستجاب الدعوه‌...,"Loved ones, who have said Twitter users answer...","Dear friends, has anyone said that Twitter use...",0.0


In [ ]:
X_train = eng[['tweet']]
Y_train = eng[['class']]


X_farsi = farsi[['text']]
Y_farsi = farsi[['ref3']]


In [ ]:
X_tune, X_test, Y_tune, Y_test = train_test_split(X_farsi, Y_farsi, test_size=0.4, random_state=101)

In [ ]:
# tokenize the dataset, truncate when passed `max_length`, 
# and pad with 0's when less than `max_length`
train_encodings = tokenizer_xlmroberta(X_train['tweet'].to_list(), truncation=True, padding=True, max_length=max_length)
farsi_valid_encodings = tokenizer_xlmroberta(X_tune['text'].to_list(), truncation=True, padding=True, max_length=max_length)

In [ ]:
class toTorchConverter(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset
train_dataset = toTorchConverter(train_encodings, Y_train['class'].to_list())
farsi_dataset_valid = toTorchConverter(farsi_valid_encodings, Y_tune['ref3'].astype(dtype="long").to_list())

In [ ]:
model = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=2).to("cuda")

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [ ]:
training_args = TrainingArguments(
    output_dir='./results_roberto',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs_roberto',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=400,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=farsi_dataset_valid,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
400,0.569300,1.129233,0.537548,33.916200,91.873000
800,0.458100,0.475772,0.839859,33.901500,91.913000
1200,0.428600,0.531694,0.844031,33.770700,92.269000
1600,0.402500,0.499226,0.814827,33.803900,92.179000
2000,0.345700,0.586893,0.739089,33.977900,91.707000


TrainOutput(global_step=2037, training_loss=0.43913782866023365, metrics={'train_runtime': 2359.4932, 'train_samples_per_second': 0.863, 'total_flos': 1.6077902822689824e+16, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 90651499, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 1041335, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1592921, 'train_mem_gpu_alloc_delta': 4497489408, 'train_mem_cpu_peaked_delta': 768919005, 'train_mem_gpu_peaked_delta': 4501180416})

In [ ]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_accuracy': 0.8398587933247753,
 'eval_loss': 0.4757724106311798,
 'eval_mem_cpu_alloc_delta': 114656,
 'eval_mem_cpu_peaked_delta': 231069,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 135332352,
 'eval_runtime': 32.7333,
 'eval_samples_per_second': 95.194}

In [ ]:
model_path = "drive/MyDrive/xlmr"
model.save_pretrained(model_path)
tokenizer_xlmroberta.save_pretrained(model_path)

('drive/MyDrive/xlmr/tokenizer_config.json',
 'drive/MyDrive/xlmr/special_tokens_map.json',
 'drive/MyDrive/xlmr/sentencepiece.bpe.model',
 'drive/MyDrive/xlmr/added_tokens.json')

In [ ]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer_xlmroberta(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return probs.argmax().item()

In [ ]:
X_test_result = [get_prediction(item) for item in X_test['text'].to_list()]


In [ ]:
print(classification_report(Y_test, X_test_result))

              precision    recall  f1-score   support

         0.0       0.88      0.83      0.86      1786
         1.0       0.24      0.32      0.27       292

    accuracy                           0.76      2078
   macro avg       0.56      0.58      0.57      2078
weighted avg       0.79      0.76      0.78      2078



In [ ]:
model_without_english = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=2).to("cuda")

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [ ]:
training_args_without_english = TrainingArguments(
    output_dir='./xlmr_without_english',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./xlmr_without_english_logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=400,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

In [ ]:
trainer_without_english = Trainer(
    model=model_without_english,                         # the instantiated Transformers model to be trained
    args=training_args_without_english,                  # training arguments, defined above
    train_dataset=farsi_dataset_valid,         # training dataset
    eval_dataset=farsi_dataset_valid,
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [ ]:
!rm -rf ./results_roberto/*

In [ ]:
trainer_without_english.train()

Step,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
400,0.431100,0.342772,0.882221,34.001900,91.642000


TrainOutput(global_step=585, training_loss=0.39083526480911124, metrics={'train_runtime': 472.9147, 'train_samples_per_second': 1.237, 'total_flos': 2744719717056768.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 49480, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 18274, 'init_mem_gpu_peaked_delta': 1112325632, 'train_mem_cpu_alloc_delta': 1111296, 'train_mem_gpu_alloc_delta': 4476404224, 'train_mem_cpu_peaked_delta': 768213493, 'train_mem_gpu_peaked_delta': 1620997120})

In [ ]:
trainer_without_english.evaluate()

{'epoch': 3.0,
 'eval_accuracy': 0.8822207958921694,
 'eval_loss': 0.34277206659317017,
 'eval_mem_cpu_alloc_delta': 120538,
 'eval_mem_cpu_peaked_delta': 226341,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 135332352,
 'eval_runtime': 32.7263,
 'eval_samples_per_second': 95.214}

In [ ]:
def get_prediction_wighout_english(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer_xlmroberta(text, padding=True, truncation=True, max_length=max_length, return_tensors="pt").to("cuda")
    # perform inference to our model
    outputs = model_without_english(**inputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    # executing argmax function to get the candidate label
    return probs.argmax().item()

In [ ]:
X_test_result = [get_prediction_wighout_english(item) for item in X_test['text'].to_list()]

print(classification_report(Y_test, X_test_result))

              precision    recall  f1-score   support

         0.0       0.91      0.97      0.94      1786
         1.0       0.68      0.43      0.53       292

    accuracy                           0.89      2078
   macro avg       0.79      0.70      0.73      2078
weighted avg       0.88      0.89      0.88      2078



In [ ]:
model_path = "drive/MyDrive/xlmr_withoutout_english"
model_without_english.save_pretrained(model_path)
tokenizer_xlmroberta.save_pretrained(model_path)

('drive/MyDrive/xlmr_withoutout_english/tokenizer_config.json',
 'drive/MyDrive/xlmr_withoutout_english/special_tokens_map.json',
 'drive/MyDrive/xlmr_withoutout_english/sentencepiece.bpe.model',
 'drive/MyDrive/xlmr_withoutout_english/added_tokens.json')